In [1]:
import urllib
import os 
import re
import tqdm
import importlib,sys
import pandas as pd
importlib.reload(sys)
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal, LAParams
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed

In [2]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

# 要抽取的字段

1、姓名  
2、出生年月  
3、性别  
4、电话  
5、最高学历  
6、籍贯  
7、落户市县  
8、政治面貌  
9、毕业院校  
10、工作单位  
11、工作内容  
12、职务  
13、项目名称  
14、项目责任  
15、学位  
16、毕业时间  
17、工作时间  
18、项目时间

# 评价指标

<img src="评估指标1.png" >

<img src="评估指标2.png" />

# pdfminer

In [3]:
 def parse_pdf(id):
 
    path = './tcdata/test_data'
    pdf_path = r'{}/{}.pdf'.format(path, id)
    
    with open(pdf_path, 'rb') as f:
        # 建立文档分析器并关联文档    
        parser = PDFParser(f)
        doc = PDFDocument()
        parser.set_document(doc)
        doc.set_parser(parser)

        #提供初始化密码，没有默认为空
        doc.initialize()

        #检查文档是否可以转成TXT，如果不可以就忽略
        if not doc.is_extractable:
            raise PDFTextExtractionNotAllowed
        else:

            #创建PDF资源管理其，来管理共享资源
            rsrcmagr = PDFResourceManager()
            laparams = LAParams()

            #创建一个聚合器，姐手资源管理器和参数
            device = PDFPageAggregator(rsrcmagr, laparams=laparams)

            #创建一个解释器，姐手资源管理器和聚合器
            interpreter = PDFPageInterpreter(rsrcmagr, device)

            #循环遍历列表，每次处理一个page内容
            #文本获取

            content = []

            for page in doc.get_pages():
                interpreter.process_page(page) # 解释器对页面进行逐一聚合
                layout = device.get_result() # 然后调用聚合器的get_result()获取到layout
                #这里的layout是一个LTPage对象 里面存放着page解析出来的各种对象
                #一般包括LTTextBox，LTFigure，LTImage，LTTextBoxHorizontal等等一些对像
                #想要获取文本就得获取对象的text属性       
                for x in layout:
                    if(isinstance(x, LTTextBoxHorizontal)):
                        result = x.get_text()
                        result =result.replace('\n', '')
                        content.append(result)

    return "".join(content)
 



In [4]:
print(parse_pdf('019ba53df6c6').replace(" ", ""))

FileNotFoundError: [Errno 2] No such file or directory: './tcdata/test_data/019ba53df6c6.pdf'

In [ ]:
# print(train_data['0a2df74bbc31'])
beautiful_format = json.dumps(train_data['0305cc4988ac'], indent=0, ensure_ascii=False)
print(beautiful_format)

In [ ]:
tag_list = ['B-PRO-NAME', 'I-PRO-NAME', 'E-PRO-NAME', 
            'B-PRO-TIME', 'I-PRO-TIME', 'E-PRO-TIME',
            'B-PRO-DUTY', 'I-PRO-DUTY', 'E-PRO-DUTY', 
            'B-GRAD-SCHOOL', 'I-GRAD-SCHOOL', 'E-GRAD-SCHOOL',
            'B-GRAD-DEGREE', 'I-GRAD-DEGREE', 'E-GRAD-DEGREE',
            'B-GRAD-TIME', 'I-GRAD-TIME', 'E-GRAD-TIME',
            'B-JOB-UNIT', 'I-JOB-UNIT', 'E-JOB-UNIT',
            'B-JOB-TIME', 'I-JOB-TIME', 'E-JOB-TIME',
            'B-JOB-CONTENT', 'I-JOB-CONTENT', 'E-JOB-CONTENT',
            'B-JOB-DUTY', 'I-JOB-DUTY', 'E-JOB-DUTY',
           ]

# train_json

In [ ]:
import json

path = './[new] train_data_20200207.json'
with open(path, 'r', encoding='utf-8') as f:
    train_data = json.load(f)

# data_process

## get_BIEO_label()

In [ ]:
def get_BIEO_label(df_word_tag, word, content, tag_name):
    index = word.find(content)
    if index > 0:
        B = 'B-' + tag_name
        I   = 'I-'  + tag_name
        E = 'E-' + tag_name
        df_word_tag.iloc[index:index+len(content),1] = [B] + [I]*(len(content)-2) + [E]
        return  df_word_tag
    else:
        return df_word_tag
 

## get_train_txt()

In [ ]:
def get_train_txt(id):
    word = parse_pdf(id).replace(" ", "")
    df_word_tag = pd.DataFrame(list(word), columns=['word'])
    df_word_tag['tag'] = 'O'

    t_dict = train_data[id]

    if '籍贯' in t_dict.keys():
        content = t_dict['籍贯']
        df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                     'NATIVE-PLACE')

    if '出生年月' in t_dict.keys():
        content = t_dict['出生年月']
        df_word_tag = get_BIEO_label(df_word_tag, word, content, 'BIRTH')

    if '项目经历' in t_dict.keys():
        l = len(t_dict['项目经历'])
        for i in range(l):
            detail = t_dict['项目经历'][i]
            if '项目名称' in detail.keys():
                content = detail['项目名称']
                df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                             'PRO-NAME')
            if '项目时间' in detail.keys():
                content = detail['项目时间']
                df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                             'PRO-TIME')
            if '项目责任' in detail.keys():
                content = detail['项目责任']
                df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                             'PRO-DUTY')

    if '教育经历' in t_dict.keys():
        l = len(t_dict['教育经历'])
        for i in range(l):
            detail = t_dict['教育经历'][i]
            if '毕业院校' in detail.keys():
                content = detail['毕业院校']
                df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                             'GRAD-SCHOOL')
            if '学位' in detail.keys():
                content = detail['学位']
                df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                             'GRAD-DEGREE')
            if '毕业时间' in detail.keys():
                content = detail['毕业时间']
                df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                             'GRAD-TIME')

    if '工作经历' in t_dict.keys():
        l = len(t_dict['工作经历'])
        for i in range(l):
            detail = t_dict['工作经历'][i]
            if '工作单位' in detail.keys():
                content = detail['工作单位']
                df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                             'JOB-UNIT')
            if '工作时间' in detail.keys():
                content = detail['工作时间']
                df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                             'JOB-TIME')
            if '工作内容' in detail.keys():
                content = detail['工作内容']
                df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                             'JOB-CONTENT')
            if '职务' in detail.keys():
                content = detail['职务']
                df_word_tag = get_BIEO_label(df_word_tag, word, content,
                                             'JOB-DUTY')

    df_word_tag = df_word_tag.append(blank_line)

    return df_word_tag


# print(tag)

## 生成train.txt的主程序

In [74]:
import random

blank_line = pd.DataFrame({'word':['index'], 'tag':['index']})

files = os.listdir(r'./tcdata/test_data/')
ids = [each.split('.')[0] for each in files]

record = 1
for id in ids:
    print(id, 'begin!')
    print(record)
    record += 1
    df_word_tag = get_train_txt(id)
    df_word_tag.to_csv(r'./train.txt', mode='a', index=False, header=False, sep=' ')

0043e770a330 begin!
1
0052b7958e89 begin!
2
0141c76d878b begin!
3
0176e4a20c37 begin!
4
01965d4b2d21 begin!
5
019ba53df6c6 begin!
6
0202e64d61db begin!
7
020b0120c240 begin!
8
021e3add6cf5 begin!
9
0224bcd05bf5 begin!
10
022e188b452d begin!
11
022f8de77ec4 begin!
12
0255dafc78a0 begin!
13
025e40bd024d begin!
14
02b5bd8a71b6 begin!
15
02dd7502bcb7 begin!
16
0305cc4988ac begin!
17
03759e1a3880 begin!
18
039482798393 begin!
19
03a7a696d8e5 begin!
20
03af1ce17ac3 begin!
21
04072b374ef8 begin!
22
0448c519f7f1 begin!
23
046794ddb166 begin!
24
047d7b917f34 begin!
25
04891776b0b6 begin!
26
04ac2fe5bb65 begin!
27
05994a53f0b3 begin!
28
05b28fa19cb6 begin!
29
05be088302e8 begin!
30
05c80876b392 begin!
31
05cbbcac2dd7 begin!
32
060eccbbf715 begin!
33
06ea442190e9 begin!
34
070546be3284 begin!
35
0718130c5820 begin!
36
076d17494064 begin!
37
078175717ca0 begin!
38
0794f8a96ad7 begin!
39
07a439a2b011 begin!
40
07a8054989e6 begin!
41
07d849fd6c53 begin!
42
07e2b5de5a50 begin!
43
07fea6bf497b begin!


5e2178fe183b begin!
688
5e3b0d134a8e begin!
689
5e739f61d0c8 begin!
690
5e842181ea40 begin!
691
5f3a69836be2 begin!
692
5f47d18ab633 begin!
693
5f509e12beff begin!
694
5f5123fbe3d5 begin!
695
5f5dc8cf1317 begin!
696
5f7a5352ae79 begin!
697
5f951bcf9300 begin!
698
5fa09dd9a6d6 begin!
699
5ffee660c1dd begin!
700
602513cd4ada begin!
701
60bc4f959686 begin!
702
60c1474fa050 begin!
703
60e5a0e51077 begin!
704
60fb98886b72 begin!
705
610cb6962d52 begin!
706
6116db66981b begin!
707
611d7cde02ed begin!
708
612c0dce6d72 begin!
709
6131b35cabc3 begin!
710
614de33a96f2 begin!
711
61515194f9a3 begin!
712
61780c9ec437 begin!
713
6197f43db272 begin!
714
61c5ecfffd52 begin!
715
61d1137d15e0 begin!
716
62059ea235a1 begin!
717
62065bbbf7f5 begin!
718
622e06ee3979 begin!
719
623f5573f130 begin!
720
6272582c2d43 begin!
721
628b630505a1 begin!
722
62a229704eb8 begin!
723
62b77d9c4a85 begin!
724
62d93cc539b3 begin!
725
63051aecde00 begin!
726
6310e79ce7bf begin!
727
632cfb94068d begin!
728
6330df2dd55e beg

af830821e634 begin!
1355
affeb7f688f1 begin!
1356
afffe2f8278a begin!
1357
b00d816bd4f8 begin!
1358
b04066076c0f begin!
1359
b051487289e8 begin!
1360
b06e7170303d begin!
1361
b0a9fc61ebd5 begin!
1362
b0f5dc92519e begin!
1363
b1334bb6f2b6 begin!
1364
b1473eae0c90 begin!
1365
b1569c38d5a5 begin!
1366
b16c9c72bed7 begin!
1367
b199d5a241a6 begin!
1368
b1b3cf6107b9 begin!
1369
b1e6776afd5f begin!
1370
b1ea865e0dd1 begin!
1371
b1ffd2e05e3c begin!
1372
b22072cc859d begin!
1373
b225f8ea4018 begin!
1374
b2ad0729bf20 begin!
1375
b2dad314d82a begin!
1376
b2e9093def9c begin!
1377
b2f6a42ba190 begin!
1378
b30a19a51f6a begin!
1379
b37b6c95e981 begin!
1380
b3bf598a0ae7 begin!
1381
b3dc19ca30f7 begin!
1382
b3eeb19f462a begin!
1383
b3f53bf78f3d begin!
1384
b40ec244dcfd begin!
1385
b415070e57cf begin!
1386
b436a6454552 begin!
1387
b4502e04b54e begin!
1388
b46b5e3753c3 begin!
1389
b488023210f8 begin!
1390
b4eddb759aec begin!
1391
b57b0e777e46 begin!
1392
b59668cdb44d begin!
1393
b5a293d0162b begin!
1394


In [5]:
import random

blank_line = pd.DataFrame({'word':['index'], 'tag':['index']})

files = os.listdir(r'./tcdata/test_data/')
ids = [each.split('.')[0] for each in files]

FileNotFoundError: [Errno 2] No such file or directory: './tcdata/test_data/'